In [1]:
!pip install pymupdf easyocr opencv-python pillow # easy ocr
!pip install pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 72.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [2]:
import io
import os
import re
import cv2
import fitz  # PyMuPDF
import gdown
import torch
import zipfile
import pickle
import easyocr
import pymupdf4llm
import pandas as pd
import numpy as np
from PIL import Image

from langchain.schema import Document

### 기본 Utils (basic_utils.py)

In [3]:
def load_secret(key_name: str):

    # 1) Kaggle 환경: os.environ 또는 파일에서 읽기
    if key_name in os.environ:
        return os.environ[key_name]

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        return user_secrets.get_secret(key_name)
    except ImportError:
        pass

    # 2) Colab 환경인지 확인
    try:
        import google.colab.userdata as userdata
        try:
            return userdata.get(key_name)
        except KeyError:
            pass # Key not in Colab userdata
    except ImportError:
        pass

    raise KeyError(f"Secret '{key_name}' not found in Colab userdata, os.environ, or file.")


def download(id, filename):
    if os.path.isfile(filename) and os.path.getsize(filename) > 0:
        print(f"[skip] {filename} 이미 존재합니다.")
    else:
        gdown.download(id=id, output=filename, quiet=False)
        print(f'[ok] {filename} 다운로드 완료.')

        if filename.lower().endswith('.zip'):
            try:
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(os.path.dirname(filename) or '.') # Extract to directory of the file, or current if no directory
                print(f'[ok] {filename} 압축풀기 완료.')
            except zipfile.BadZipFile:
                print(f"[warning] {filename}은 유효한 zip 파일이 아닙니다.")

    return filename


def load_csv(filename):
  """Loads a CSV file into a pandas DataFrame."""
  try:
    df = pd.read_csv(filename)
    print(f"[ok] {filename} 로드 완료.")
    return df
  except FileNotFoundError:
    print(f"[error] {filename} 파일을 찾을 수 없습니다.")
    return None
  except Exception as e:
    print(f"[error] {filename} 로드 중 오류 발생: {e}")
    return None


def save_pages(pages, filename):
    directory = os.path.dirname(filename)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
        print(f"디렉토리 '{directory}'를 생성했습니다.")
    with open(filename, 'wb') as f:
        pickle.dump(pages, f)
    print(f"'{filename}' 파일에 pages를 저장했습니다.")


def load_pages(filename):
    if not os.path.exists(filename):
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다.")
        return None
    with open(filename, 'rb') as f:
        pages = pickle.load(f)
    print(f"'{filename}' 파일에서 pages를 로드했습니다.")
    return pages

def ext(original_filename, ext='pkl'):
  base_filename, _ = os.path.splitext(original_filename)
  return f"{base_filename}.{ext}"

### PDF/HWP 문서 Utils (document_utils.py)

In [22]:
def load_pymupdf(pdf_path, filename, langs=['ko','en'], zoom=3.0, gpu=True):
    """
    PDF 페이지별 텍스트 레이어 + EasyOCR 결과 병합
    - langs: EasyOCR 언어 리스트
    - zoom: 해상도 배율 (3.0이면 약 216dpi)
    - gpu: GPU 사용 여부
    """
    # 캐시 로드
    output_path = f'outputs/{filename}'
    if os.path.exists(output_path):
        return load_pages(output_path)

    # EasyOCR Reader 초기화
    reader = easyocr.Reader(langs, gpu=gpu)

    doc = fitz.open(f'files/{pdf_path}')
    pages = []

    for page_num, page in enumerate(doc, start=1):
        # 1. 텍스트 레이어 추출
        text_layer = page.get_text()

        # 2. 고해상도 렌더링
        matrix = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=matrix, alpha=False)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # 3. EasyOCR 실행
        cv_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        results = reader.readtext(cv_img, detail=0)  # detail=0 → 텍스트만 리스트로 반환
        ocr_text = "\n".join(results)

        # 4. 병합
        merged_text = (text_layer or "").strip() + "\n" + (ocr_text or "").strip()

        print(f"[페이지 {page_num}] 텍스트 길이: {len(text_layer)}, OCR 길이: {len(ocr_text)}")
        pages.append(Document(page_content=merged_text))
        display(img)
        print(ocr_text)
    doc.close()

    # 캐시 저장
    save_pages(pages, output_path)
    return pages


def load_pymupdf4llm_easyocr(pdf_filename):
    IMAGE_PATH = 'images'
    pdf_path = f'files/{pdf_filename}'
    output_path = f'outputs/{ext(pdf_filename)}'
    if os.path.exists(output_path):
        return load_pages(output_path)
    
    os.makedirs(IMAGE_PATH, exist_ok=True)
    
    # EasyOCR 리더 객체 초기화 (GPU 사용 가능 시 활용)
    reader = easyocr.Reader(['ko', 'en'], gpu=torch.cuda.is_available())

    # 1. PyMuPDF4LLM을 사용해 마크다운 생성 및 이미지 추출
    # write_images=True 옵션으로 이미지 추출
    # image_path 옵션으로 이미지 저장 위치 지정
    # page_chunks=True 옵션으로 페이지별로 분리된 결과 생성
    extracted_chunks = pymupdf4llm.to_markdown(pdf_path, write_images=True, image_path=IMAGE_PATH, page_chunks=True)

    # final_markdown = ""
    pages = []
    # 2. EasyOCR로 이미지 내 텍스트 인식 후 마크다운에 추가
    for chunk in extracted_chunks:
        page_text = chunk["text"]
        
        page_number = chunk["metadata"]["page"] # Get the page index from metadata
        print('페이지: ', page_number)
        
        # 마크다운 내 이미지 참조 찾기 (예: ![image_name](images/image_name.png))
        # ([^\]]+)는 이미지 이름 부분을 캡처
        image_ref_pattern = re.compile(r"!\[(.*?)\]\((.*?)\)")
        matches = image_ref_pattern.finditer(page_text)
        
        # 이미지 참조를 순서대로 처리
        for match in reversed(list(matches)):
            image_path = match.group(2)
            
            # EasyOCR로 이미지 내 텍스트 인식
            try:
                ocr_results = reader.readtext(image_path)
                ocr_caption = " ".join([text for _, text, _ in ocr_results])
            except Exception as e:
                print(f"OCR 처리 중 오류 발생: {e}")
                ocr_caption = "OCR 처리 실패"
            
            # 마크다운 텍스트 업데이트 (이미지 참조 다음에 OCR 결과 삽입)
            original_match_string = match.group(0)
            replacement_string = f"{original_match_string}\n\n**OCR 텍스트:** {ocr_caption}\n"
            page_text = page_text[:match.start()] + replacement_string + page_text[match.end():]
        pages.append(Document(page_content=page_text))
        # final_markdown += page_text + "\n\n"

    save_pages(pages, output_path)
    return pages #final_markdown



### 필요자료 다운로드

In [5]:
download('1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb', 'output.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb
From (redirected): https://drive.google.com/uc?id=1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb&confirm=t&uuid=ca6a87e0-add0-4e18-876d-337d444ad6da
To: /kaggle/working/output.zip
100%|██████████| 157M/157M [00:01<00:00, 83.4MB/s] 


[ok] output.zip 다운로드 완료.
[ok] output.zip 압축풀기 완료.


'output.zip'

### CSV 파일 확인하기

In [6]:
df = load_csv('data_list.csv')
df.head()

[ok] data_list.csv 로드 완료.


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaN,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\r\n \r\n \r\n \r\n제 안 요 청 서\r\n[ 2024년 대학 ...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\r\n \r\nEIP3.0 고압가스 안전관리\r\n시스템 구축 용역\...
3,20240430918,0.0,도시계획위원회 통합관리시스템 구축용역,150000000.0,인천광역시,2024-04-18 16:26:32,2024-05-02 10:00:00,2024-05-09 16:00:00,- 사업명: 도시계획위원회 통합관리시스템 구축 용역\n- 용역개요: 도시계획위원회와...,hwp,인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp,\r\n \r\n \r\n도시계획위원회 통합관리시스템 구축\r\n제 안 요 청...
4,20240430896,0.0,봉화군 재난통합관리시스템 고도화 사업(협상)(긴급),900000000.0,경상북도 봉화군,2024-04-18 16:33:28,2024-04-26 09:00:00,2024-04-30 17:00:00,- 사업명: 봉화군 재난통합관리시스템 고도화 사업\n- 사업개요: 공동수급(공동이행...,hwp,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).hwp,\r\n \r\n \r\n제안요청서\r\n \r\n사 업 명\r\n봉화...


### PDF 샘플 로딩

In [29]:
pdf_df = df[df['파일형식'] == 'pdf']
pdf_filename = pdf_df.iloc[0]['파일명'] # 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
pages = load_pymupdf4llm_easyocr(pdf_filename) # 추후 pages를 반환하도록 변경해야 함.
markdown_output = '\n\n---\n\n'.join([p.page_content for p in pages])
with open(ext(pdf_filename, 'md'), "w", encoding="utf-8") as f:
    f.write(markdown_output)
print(f"PDF 처리 완료. {ext(pdf_filename, 'md')} 파일 생성.")

페이지:  1
페이지:  2
페이지:  3
페이지:  4
페이지:  5
페이지:  6
페이지:  7
페이지:  8
페이지:  9
페이지:  10
페이지:  11
페이지:  12
페이지:  13
페이지:  14
페이지:  15
페이지:  16
페이지:  17
페이지:  18
페이지:  19
페이지:  20
페이지:  21
페이지:  22
페이지:  23
페이지:  24
페이지:  25
페이지:  26
페이지:  27
페이지:  28
페이지:  29
페이지:  30
페이지:  31
페이지:  32
페이지:  33
페이지:  34
페이지:  35
페이지:  36
페이지:  37
페이지:  38
페이지:  39
페이지:  40
페이지:  41
페이지:  42
페이지:  43
페이지:  44
페이지:  45
페이지:  46
페이지:  47
페이지:  48
페이지:  49
페이지:  50
페이지:  51
페이지:  52
페이지:  53
페이지:  54
페이지:  55
페이지:  56
페이지:  57
페이지:  58
페이지:  59
페이지:  60
페이지:  61
페이지:  62
페이지:  63
페이지:  64
페이지:  65
페이지:  66
페이지:  67
페이지:  68
페이지:  69
페이지:  70
페이지:  71
페이지:  72
페이지:  73
페이지:  74
페이지:  75
페이지:  76
페이지:  77
페이지:  78
페이지:  79
페이지:  80
페이지:  81
페이지:  82
페이지:  83
페이지:  84
페이지:  85
페이지:  86
페이지:  87
페이지:  88
페이지:  89
페이지:  90
페이지:  91
페이지:  92
페이지:  93
페이지:  94
페이지:  95
페이지:  96
페이지:  97
페이지:  98
페이지:  99
페이지:  100
페이지:  101
페이지:  102
페이지:  103
페이지:  104
페이지:  105
페이지:  106
페이지:  107
페이지:  108
페이지:  109
페이지:  110
페이지:  11

### HWP 샘플 로딩

In [19]:
# 예시 사용
hwp_name = df.iloc[0]['파일명'] # 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
print(f"원본 파일명: {hwp_name}")
print(f".pkl 파일명: {ext(hwp_name)}")

원본 파일명: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
.pkl 파일명: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.pkl
